<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MistralLarge_Gpt4_AAI_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-openai -q
!pip install colab-env -q
!pip install mistralai -q
!pip install openai-agents -q
!pip install langchain-mistralai -q
!pip install pydantic -q
!pip install crewai -q
!pip install crewai_tools -q
!pip install crewai['tools'] -q

In [2]:
import colab_env

Mounted at /content/gdrive


## AGENT

In [1]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain_mistralai import ChatMistralAI
from pydantic import BaseModel, Field
from typing import List
from dotenv import load_dotenv
from crewai.tools import tool
import colab_env

# Load environment variables from .env file
load_dotenv()

# --- 0. Set up Environment Variables ---
openai_api_key = os.environ.get("OPENAI_API_KEY")
mistral_api_key = os.environ.get("MISTRAL_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set. Please set it.")
if not mistral_api_key:
    raise ValueError("MISTRAL_API_KEY environment variable not set. Please set it.")

print("API keys loaded successfully.")

# --- 1. Define Tools (Now properly wrapped with @tool) ---
def _calculate_fuel_burn_logic(aircraft_type: str, distance_km: float, weather_conditions: str) -> float:
    """Helper function with the core logic for fuel burn calculation."""
    base_fuel_rate_per_km = 0.05
    if "Boeing 787" in aircraft_type:
        base_fuel_rate_per_km = 0.08

    fuel_burn = distance_km * base_fuel_rate_per_km

    if "high winds" in weather_conditions.lower() or "thunderstorms" in weather_conditions.lower():
        fuel_burn *= 1.15 # 15% increase for adverse weather
    return fuel_burn

@tool("Get Weather Forecast")
def get_weather_forecast(location: str, time: str) -> str:
    """Fetches detailed weather forecast for a given location and time."""
    print(f" [Tool:get_weather_forecast] Fetching weather for {location} at {time}...")
    if "storm" in location.lower() or "bad weather" in time.lower():
        return "Severe thunderstorms with high winds and low visibility expected. Instrument Flight Rules (IFR) conditions."
    return "Clear skies, light winds, good visibility. Visual Flight Rules (VFR) conditions."

@tool("Get NOTAMs")
def get_notams(airspace: str) -> str:
    """Retrieves Notices to Airmen (NOTAMs) for a specified airspace."""
    print(f" [Tool:get_notams] Retrieving NOTAMS for {airspace}...")
    if "LAX" in airspace.upper():
        return "NOTAM: LAX Runway 25L closed for maintenance until 2025-07-15. Expect delays."
    if "LONDON HEATHROW" in airspace.upper() or "EGLL" in airspace.upper():
        return "NOTAM: EGLL ATC system experiencing minor disruptions, expect minor flow control."
    return "No significant NOTAMS for this airspace at this time."

@tool("Check Regulatory Compliance")
def check_regulatory_compliance(route_details: str, regulations: List[str]) -> str:
    """Checks a proposed route against aviation regulations and provides detailed reasoning."""
    print(f" [Tool:check_regulatory_compliance] Checking regulatory compliance for route: '{route_details}' against regulations: {', '.join(regulations)}...")
    compliance_issues = []
    reasoning_steps = []

    if "restricted airspace" in route_details.lower() and not "special clearance" in route_details.lower():
        compliance_issues.append("Violation: Route enters restricted airspace without explicit clearance.")
        reasoning_steps.append("Identified 'restricted airspace' in route details. No mention of 'special clearance'.")

    # This is a hypothetical call and should be replaced with actual context passing
    current_weather_at_dest = "Clear skies, light winds, good visibility. Visual Flight Rules (VFR) conditions."
    if "ICAO Annex 2" in regulations and "VFR" in route_details and "IFR" in current_weather_at_dest.lower():
        compliance_issues.append("Violation: Attempting Visual Flight Rules (VFR) in anticipated Instrument Flight Rules (IFR) conditions at destination.")
        reasoning_steps.append("Checked ICAO Annex 2. VFR flights require specific weather minima. Destination weather indicates IFR.")

    if not compliance_issues:
        return f"Compliance Check: PASSED. All specified regulations ({', '.join(regulations)}) appear to be met for the route '{route_details}'."
    else:
        return f"Compliance Check: FAILED.\nIssues: {'; '.join(compliance_issues)}\nDetailed Reasoning: {'; '.join(reasoning_steps)}"

@tool("Calculate Fuel Burn")
def calculate_fuel_burn(aircraft_type: str, distance_km: float, weather_conditions: str) -> str:
    """Calculates estimated fuel burn for a given aircraft type, distance and weather."""
    print(f" [Tool:calculate_fuel_burn] Calculating fuel burn for {aircraft_type} over {distance_km:.0f}km with '{weather_conditions}'...")
    fuel_burn = _calculate_fuel_burn_logic(aircraft_type, distance_km, weather_conditions)
    return f"Estimated fuel burn: {fuel_burn:.2f} liters."

@tool("Calculate Flight Cost")
def calculate_flight_cost(distance_km: float, fuel_price_per_liter: float, aircraft_type: str) -> str:
    """Calculates the total flight cost including fuel, landing fees, and navigation fees."""
    print(f" [Tool:calculate_flight_cost] Calculating cost for {distance_km:.0f}km flight...")

    # Use existing fuel burn logic
    weather_conditions = "Clear skies, light winds, good visibility." # Assume standard conditions for cost estimate
    fuel_burn_liters = _calculate_fuel_burn_logic(aircraft_type, distance_km, weather_conditions)

    fuel_cost = fuel_burn_liters * fuel_price_per_liter

    # Hypothetical fees
    landing_fees = 5000
    navigation_fees = distance_km * 0.1

    total_cost = fuel_cost + landing_fees + navigation_fees

    return (f"**Final Flight Cost Estimation:**\n\n"
            f"*   **Fuel Cost:** ${fuel_cost:,.2f} ({fuel_burn_liters:,.2f} liters at ${fuel_price_per_liter}/liter)\n"
            f"*   **Landing Fees:** ${landing_fees:,.2f}\n"
            f"*   **Navigation Fees:** ${navigation_fees:,.2f}\n"
            f"--- \n"
            f"**Total Estimated Cost:** **${total_cost:,.2f}**")


# --- 2. Configure LLMs for CrewAI ---
openai_llm = ChatOpenAI(model_name="gpt-4o", api_key=openai_api_key)
print("OpenAI LLM (gpt-4o) configured for CrewAI.")

mistral_llm = ChatMistralAI(model="mistral/mistral-large-latest", api_key=mistral_api_key)
print("Mistral LLM (mistral-large-latest) configured for CrewAI.")


# --- 3. Define Agents ---
route_optimizer_agent = Agent(
    role='Expert Flight Route Optimizer',
    goal='Propose the most efficient and safe flight path considering weather and fuel.',
    backstory=(
        "You are a seasoned aeronautical engineer and a master of route planning."
        "You analyze weather patterns, aircraft performance, and historical data"
        "to craft the optimal flight path for any given journey."
    ),
    llm=openai_llm,
    tools=[get_weather_forecast, calculate_fuel_burn],
    verbose=False,
    allow_delegation=False
)
print('\n')
print("RouteOptimizerAgent defined with GPT-4o.")

regulatory_risk_agent = Agent(
    role='Aviation Regulatory and Risk Analyst',
    goal='Meticulously review proposed flight routes for compliance and identify potential risks.',
    backstory=(
        "You are a meticulous aviation law expert and a sharp-eyed risk assessor."
        "Your expertise lies in dissecting flight plans against a myriad of regulations"
        "and identifying any potential hazards or non-compliance issues."
    ),
    llm=mistral_llm,
    tools=[get_notams, check_regulatory_compliance],
    verbose=False,
    allow_delegation=False,
    mode='ReAct'
)
print("RegulatoryRiskAgent defined with Mistral Large.")

cost_analysis_agent = Agent(
    role='Cost Analysis Expert',
    goal='Calculate the total cost of the flight, including fuel, fees, and other operational expenses.',
    backstory=(
        "An expert in aviation finance, skilled at breaking down the complex costs "
        "associated with a flight to provide a clear and accurate budget."
    ),
    llm=openai_llm,
    tools=[calculate_flight_cost],
    verbose=False,
    allow_delegation=False,
    mode='ReAct'
)
print("CostAnalysisAgent defined with GPT-4o.")

reporting_agent = Agent(
    role='Flight Plan Reporter',
    goal='Create a comprehensive final report that combines the route, cost, and compliance analysis.',
    backstory=(
        "A skilled technical writer with expertise in aviation, responsible for compiling "
        "and presenting the final flight plan in a clear, concise, and easy-to-understand format."
    ),
    llm=openai_llm,
    verbose=False,
    allow_delegation=False
)
print("ReportingAgent defined with GPT-4o.")


# --- 4. Define Tasks ---
optimize_route_task = Task(
    description=(
        "Propose the most efficient and safe flight route from {origin} to {destination} "
        "for a {aircraft_type} aircraft. "
        "Consider current weather at both locations and estimate fuel burn. "
        "The output must include:\n"
        "1. Proposed Route Details (e.g., waypoints, altitude).\n"
        "2. Detailed Reasoning for the route, altitude, and fuel estimation.\n"
        "3. Estimated Fuel Burn in liters.\n"
        "4. Estimated Time of Arrival (ETA)."
    ),
    expected_output="A comprehensive flight route proposal including route details, reasoning, estimated fuel burn, and ETA.",
    agent=route_optimizer_agent,
    output_file='initial_route_proposal.md'
)
print('\n')
print("OptimizeRoute task defined.")

calculate_cost_task = Task(
    description=(
        "Calculate the total operational cost for the proposed flight route from the context. "
        "Use a fuel price of ${fuel_price_per_liter} per liter. "
        "The output must be a detailed breakdown of costs, including fuel, landing, and navigation fees."
    ),
    expected_output="A detailed cost breakdown including total fuel cost, operational fees, and the grand total cost.",
    agent=cost_analysis_agent,
    context=[optimize_route_task],
    output_file='flight_cost_report.md'
)
print("CalculateCost task defined.")

check_compliance_task = Task(
    description=(
        "Review the proposed flight route for full compliance with ICAO Annex 2 "
        "and national airspace regulations. Assess all potential risks, considering "
        "current NOTAMs for {origin} and {destination}."
        "Provide a transparent, step-by-step justification for every finding. "
        "If the route is compliant, state 'Compliance Check: PASSED'. "
        "If not, detail all 'Compliance Check: FAILED' issues and provide detailed reasoning."
    ),
    expected_output="A detailed compliance and risk assessment report. Clearly state PASSED or FAILED with justifications.",
    agent=regulatory_risk_agent,
    context=[optimize_route_task, calculate_cost_task],
    output_file='final_compliance_report.md'
)
print("CheckCompliance task defined.")

reporting_task = Task(
    description=(
        "Compile a comprehensive final report that includes the optimized route proposal, "
        "the detailed cost analysis, and the final compliance and risk assessment. "
        "The report should be well-structured, easy to read, and present all key findings "
        "in a clear and concise manner."
    ),
    expected_output="A single, consolidated report that includes the flight route, cost breakdown, and compliance check.",
    agent=reporting_agent,
    context=[optimize_route_task, calculate_cost_task, check_compliance_task],
    output_file='final_flight_plan.md'
)
print("Reporting task defined.")


# --- 5. Assemble and Run the Crew ---
flight_planning_crew = Crew(
    agents=[route_optimizer_agent, cost_analysis_agent, regulatory_risk_agent, reporting_agent],
    tasks=[optimize_route_task, calculate_cost_task, check_compliance_task, reporting_task],
    process=Process.sequential,
    verbose=False
)

print('\n')
print("Flight Planning Crew assembled.")

# --- Main execution block ---
if __name__ == "__main__":
    from datetime import datetime
    current_time = datetime.now().strftime("%A, %B %d, %Y at %I:%M:%S %p %Z")
    print(f"\n--- Initiating Flight Planning Workflow ({current_time}) ---")

    flight_inputs = {
        'origin': "Montreal, QC, Canada",
        'destination': "London, UK",
        'aircraft_type': "Boeing 787 Dreamliner",
        'fuel_price_per_liter': 1.50
    }

    print(f"\nPlanning flight from {flight_inputs['origin']} to {flight_inputs['destination']} "
          f"with {flight_inputs['aircraft_type']}.\n")

    final_result = flight_planning_crew.kickoff(inputs=flight_inputs)

    print("\n--- Flight Planning Workflow Completed ---")
    print("\nFinal Flight Planning Report:\n")
    print(final_result)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
API keys loaded successfully.
OpenAI LLM (gpt-4o) configured for CrewAI.
Mistral LLM (mistral-large-latest) configured for CrewAI.


RouteOptimizerAgent defined with GPT-4o.
RegulatoryRiskAgent defined with Mistral Large.
CostAnalysisAgent defined with GPT-4o.
ReportingAgent defined with GPT-4o.


OptimizeRoute task defined.
CalculateCost task defined.
CheckCompliance task defined.
Reporting task defined.


Flight Planning Crew assembled.

--- Initiating Flight Planning Workflow (Friday, August 01, 2025 at 05:06:18 PM ) ---

Planning flight from Montreal, QC, Canada to London, UK with Boeing 787 Dreamliner.

 [Tool:get_weather_forecast] Fetching weather for Montreal, QC, Canada at current...
 [Tool:get_weather_forecast] Fetching weather for London, UK at current...
 [Tool:calculate_fuel_burn] Calculating fuel burn for Boeing 787 Dreamliner over 5220km with 